In [ ]:
import h5py
import os                                 # pour manipuler et aller chercher les fichiers en local
import numpy as np                        # pour effectuer des opérations sur les array/tableaux
import scipy.io as sio                    # pour transformer les dictionnaires .mat en type numpy.ndarray

def matlab_to_numpy(path): 
  chemin = os.path.abspath(path)
  dossier = [img for img in os.listdir(chemin) if os.path.isfile(os.path.join(chemin, img))] # noms des fichiers
  # sauvegarde en numpy
  I = len(dossier)
  data = np.empty(I, dtype = object)
  for i in range(I):
    try:
      image_mat = h5py.File(os.path.join(chemin, dossier[i]), 'r')
      data[i] = np.array(image_mat['BigData'])
      data[i] = data[i].transpose(3, 2, 1, 0)
    except:
      image_mat = sio.loadmat(os.path.join(chemin, dossier[i]))
      data[i] = image_mat['Data'][0][0][0]
      N, M, T = data[i].shape[0], data[i].shape[1], data[i].shape[2]
      data[i] = data[i].reshape(1, N, M, T)
  np.save('/content/drive/MyDrive/Colab Notebooks/Data numpy/bmode.npy', data)
  return data

In [ ]:
import numpy as np
import torch                              # pour construire le modèle de réseaux de neurones
from torch.autograd import Variable
def numpy_to_tensor(path):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  chemin = os.path.abspath(path)
  data = np.load(chemin, allow_pickle=True)
  for i in range(len(data)):
    if data[i].max() > 1:
      if data[i].ndim > 3:
        B, N, M, T = data[i].shape
        m = int(M/2)
        data[i] = data[i][:,:,m,:]
        data[i] = Variable(torch.from_numpy(data[i]/255.0).reshape(data[i].shape[0], 1, N, T)).type(torch.FloatTensor)
      else:
        B, N, M = data[i].shape
        tensor = Variable(torch.from_numpy(data[i]/255.0).reshape(data[i].shape[0], 1, N, M)).type(torch.FloatTensor)
    else:
      if data[i].ndim > 3:
        B, N, M, T = data[i].shape
        m = int(M/2)
        data[i] = data[i][:,:,m,:]
        data[i] = Variable(torch.from_numpy(data[i]).reshape(data[i].shape[0], 1, N, T)).type(torch.FloatTensor)
      else:
        B, N, M = data[i].shape
        data[i] = Variable(torch.from_numpy(data[i]).reshape(data[i].shape[0], 1, N, M)).type(torch.FloatTensor)
  torch.save(data, '/content/drive/MyDrive/Colab Notebooks/Data tensor/mmode_tensor.pt')
  return data

In [ ]:
import torch                              # pour construire le modèle de réseaux de neurones

def ConvUnet(K_in, K_out, ker = 3, pad = 1):
  conv = torch.nn.Sequential(
      torch.nn.Conv2d(K_in, K_out, kernel_size = ker, stride = 1, 
                      padding = pad, bias = False),
      torch.nn.BatchNorm2d(K_out),
      torch.nn.ReLU(),
      torch.nn.Conv2d(K_out, K_out, kernel_size = ker, stride = 1, 
                      padding = pad, bias = False),
      torch.nn.BatchNorm2d(K_out),
      torch.nn.ReLU(),
      #torch.nn.Dropout2d(p = 0.05),
  )
  return conv

def crop_image(img1, img2):
  N_img1 = img1.size(2)
  M_img1 = img1.size(3)

  N_img2 = img1.size(2)
  M_img2 = img2.size(3)

  N_img = N_img1 - N_img2
  N_img = N_img // 2
  M_img = M_img1 - M_img2
  M_img = M_img // 2
  return img1[:,:, N_img:N_img1 - N_img, M_img:M_img1 - M_img]

class U_Net(torch.nn.Module):

    def __init__(self):
        super(U_Net, self).__init__()

        self.pool = torch.nn.MaxPool2d(2, 2)
        C = 8

        #Encoder
        self.conv1 = ConvUnet(1, C)

        self.conv2 = ConvUnet(C, 2*C)

        self.conv3 = ConvUnet(2*C, 4*C)

        self.conv4 = ConvUnet(4*C, 8*C)

        self.conv5 = ConvUnet(8*C, 16*C)

        self.up_conv4 = torch.nn.ConvTranspose2d(16*C, 8*C, kernel_size = 2, stride = 2)
        self.conv12 = ConvUnet(16*C, 8*C)

        self.up_conv5 = torch.nn.ConvTranspose2d(8*C, 4*C, kernel_size = 2, stride = 2)
        self.conv13 = ConvUnet(8*C, 4*C)

        self.up_conv6 = torch.nn.ConvTranspose2d(4*C, 2*C, kernel_size = 2, stride = 2)
        self.conv14 = ConvUnet(4*C, 2*C)

        self.up_conv7 = torch.nn.ConvTranspose2d(2*C, C, kernel_size = 2, stride = 2)
        self.conv15 = ConvUnet(2*C, C)

        self.conv16 = torch.nn.Conv2d(C, 1, kernel_size = 1)  

    def forward(self, x):
        batch_size, Ch, N, M = x.size()
        C = 8
        
        # Permet de traiter des entrées de tailles quelconques
        k1, k2 = [], []
        Nb_conv = 4
        kernel = 2*np.ones((Nb_conv, 2), dtype=int)
        for i in range(Nb_conv):
          k1.append(int(N/2**i))
          k2.append(int(M/2**i))
          if k1[i] % 2 == 0 and k2[i] % 2 == 0:
            kernel[i,:] = np.array([2, 2])
          elif k1[i] % 2 != 0 and k2[i] % 2 == 0:
            kernel[i,:] = np.array([3, 2])
          elif k1[i] % 2 == 0 and k2[i] % 2 != 0:
            kernel[i,:] = np.array([2, 3])
          elif k1[i] % 2 != 0 and k2[i] % 2 != 0:
            kernel[i,:] = np.array([3, 3])
          else:
            kernel[i,:] = np.array([3, 3])

        # Encoder
        x1 = self.conv1(x) # 2 CROP
        pool1 = self.pool(x1) # 114x150 190x150

        x2 = self.conv2(pool1) # 2 CROP
        pool2 = self.pool(x2) # 57x75 95x75

        x3 = self.conv3(pool2) # 4 CROP
        pool3 = self.pool(x3) # 28x37 47x37

        x4 = self.conv4(pool3) # 4 CROP
        pool4 = self.pool(x4) # 14x18 23x18
        
        x5 = self.conv5(pool4) # 8 CROP

        self.up_conv4 = torch.nn.ConvTranspose2d(16*C, 8*C, kernel_size = kernel[-1,:], stride = 2).to(device)
        x18 = self.up_conv4(x5) # 28x37 47x37
        y4 = crop_image(x4, x18)
        x19 = torch.cat((y4, x18), dim = 1)
        x20 = self.conv12(x19)

        self.up_conv5 = torch.nn.ConvTranspose2d(8*C, 4*C, kernel_size = kernel[-2,:], stride = 2).to(device)
        x21 = self.up_conv5(x20) # 57x75 95x75
        y5 = crop_image(x3, x21)
        x22 = torch.cat((y5, x21), dim = 1)
        x23 = self.conv13(x22)

        self.up_conv6 = torch.nn.ConvTranspose2d(4*C, 2*C, kernel_size = kernel[-3,:], stride = 2).to(device)
        x24 = self.up_conv6(x23) # 114x150 190x150
        y6 = crop_image(x2, x24)
        x25 = torch.cat((y6, x24), dim = 1)
        x26 = self.conv14(x25)

        self.up_conv7 = torch.nn.ConvTranspose2d(2*C, C, kernel_size = kernel[-4,:], stride = 2).to(device)
        x27 = self.up_conv7(x26) # 228x330 380x300
        y7 = crop_image(x1, x27)
        x28 = torch.cat((y7, x27), dim = 1)
        x29 = self.conv15(x28)
        
        out = self.conv16(x29)
        return out


In [ ]:
import torch
def initialize_parameters(model, path):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  
  model_net = model.to(device)
  u_net = torch.load(path, map_location = torch.device('cpu'))
  u_net.eval()
  for param0, param in zip(u_net.parameters(), model_net.parameters()):
    param.data = param0.data

In [ ]:
import cv2
def superposition(img, mask, alpha, beta, gam):
  sp1 = np.empty(len(img), dtype = object)
  sp2 = np.empty(len(img), dtype = object)
  img = img[:,0,:,:300].cpu().detach().numpy()
  mask = mask[:,0].cpu().detach().numpy().astype(img.dtype)
  linear_mask = mask
  for i in range(len(img)):
    sp1[i] = cv2.addWeighted(img[i], alpha, mask[i], beta, gam)
    for v in range(mask.shape[-1]):
      x = np.where(mask[i,:,v] == 1)[0]
      for u in range(mask.shape[-2]):
        if x.min() < u < x.max():
          linear_mask[i,u,v] = 0
    sp2[i] = cv2.addWeighted(img[i], alpha, linear_mask[i], beta, gam)
  return sp1, sp2